In [3]:
import folium
import csv
import json
from shapely.geometry import shape
import pandas as pd

# 1. Load Telangana boundary GeoJSON
with open("telangana_ac.geojson", "r", encoding="utf-8") as f:
    telangana_geojson = json.load(f)

# 1.5. Load district boundaries
with open("districts.json", "r", encoding="utf-8") as f:
    districts_geojson = json.load(f)

# 1.5. Load district boundaries
with open("TS_District_Boundary_33_FINAL.geojson", "r", encoding="utf-8") as f:
    new_districts_geojson = json.load(f)

# 2. Compute bounds of Telangana for restricting map view
bounds = folium.GeoJson(telangana_geojson).get_bounds()

# 3. Create base map centered on Telangana and restricted to its bounds with clean tiles
m = folium.Map(
    location=[17.9784, 79.5941],
    zoom_start=7,
    max_bounds=True,
    tiles="CartoDB positron",
    attr='CartoDB'
)

m.fit_bounds(bounds)
Marker_coordinates = []

# 4. Create feature groups for layer control
new_district_layer = folium.FeatureGroup(name="New District Boundaries")
district_layer = folium.FeatureGroup(name="District Boundaries")
constituency_layer = folium.FeatureGroup(name="Constituency Boundaries")

# 5. Add district boundaries to district layer
for feature in districts_geojson["features"]:
    # Generate different colors for each district
    district_id = feature['properties'].get('OBJECTID', 0)
    district_name = feature['properties'].get('D_N', 'Unknown District')
    
    # Create distinct colors using HSV color space
    hue = (district_id * 137.5) % 360  # Golden angle for good distribution
    color = f"hsl({hue}, 70%, 50%)"
    darker_color = f"hsl({hue}, 70%, 30%)"  # Darker version for text
    
    folium.GeoJson(
        feature,
        style_function=lambda x, color=color: {
            "color": color,
            "weight": 3,
            "fillOpacity": 0.1
        }
    ).add_to(district_layer)
    
    # Add district name label at centroid
    try:
        geom = feature["geometry"]
        polygon = shape(geom)
        centroid = polygon.centroid
        lat, lon = centroid.y, centroid.x
        
        folium.map.Marker(
            [lat, lon],
            icon=folium.DivIcon(html=f'<div style="font-size:15pt; color:{darker_color}; font-weight:bold; text-shadow:1px 1px white">{district_name}</div>')
        ).add_to(district_layer)
    except Exception as e:
        print(f"Error computing centroid for {district_name}: {e}")

# 5. Add district boundaries to NEW district layer
i=0
for feature in new_districts_geojson["features"]:
    # Generate different colors for each district
    district_id = feature['properties'].get('OBJECTID', i)
    i += 1
    district_name = feature['properties'].get('DISTRICT_N', 'Unknown District')
    
    # Create distinct colors using HSV color space
    hue = (district_id * 137.5) % 360  # Golden angle for good distribution
    color = f"hsl({hue}, 70%, 50%)"
    darker_color = f"hsl({hue}, 70%, 30%)"  # Darker version for text
    
    folium.GeoJson(
        feature,
        style_function=lambda x, color=color: {
            "color": color,
            "weight": 3,
            "fillOpacity": 0.1
        }
    ).add_to(new_district_layer)
    
    # Add district name label at centroid
    try:
        geom = feature["geometry"]
        polygon = shape(geom)
        centroid = polygon.centroid
        lat, lon = centroid.y, centroid.x
        
        folium.map.Marker(
            [lat, lon],
            icon=folium.DivIcon(html=f'<div style="font-size:15pt; color:{darker_color}; font-weight:bold; text-shadow:1px 1px white">{district_name}</div>')
        ).add_to(new_district_layer)
    except Exception as e:
        print(f"Error computing centroid for {district_name}: {e}")

# 6. Add constituency boundaries and labels to constituency layer
for feature in telangana_geojson["features"]:
    geom = feature["geometry"]
    props = feature["properties"]
    constituency_name = props.get("assembly_1") or props.get("assembly") or "Unknown"

    folium.GeoJson(
        feature,
        style_function=lambda x: {
            "color": "#0000FF",
            "weight": 0.5,
            "fillOpacity": 0.05
        }
    ).add_to(constituency_layer)

    # Compute centroid using shapely
    try:
        polygon = shape(geom)
        centroid = polygon.centroid
        lat, lon = centroid.y, centroid.x
        Marker_coordinates.append({"constituency": constituency_name, "latitude": lat, "longitude": lon})
        folium.map.Marker(
            [lat, lon],
            icon=folium.DivIcon(html=f'<div style="font-size:10pt; color:blue; text-shadow:1px 1px black">{constituency_name}</div>')
        ).add_to(constituency_layer)
    except Exception as e:
        print(f"Error computing centroid for {constituency_name}: {e}")

# 7. Add layers to map
district_layer.add_to(m)
constituency_layer.add_to(m)
new_district_layer.add_to(m)

# 8. Add layer control
folium.LayerControl().add_to(m)

# 5. Load MLA data from CSV
with open("mlas.csv", newline='', encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        try:
            lat = float(row["latitude"])
            lon = float(row["longitude"])
            name = row["name"]
            constituency = row["constituency"]
            party = row["party"]
            photo_url = row["photo_url"]

            # Popup HTML
            popup_html = f"""
            <div style=\"text-align:center; width:180px;\">
                <img src=\"{photo_url}\" style=\"width:100px; height:100px; border-radius:8px; margin-bottom:10px;\"><br>
                <strong>{name}</strong><br>
                <em>{constituency}</em><br>
                <span>{party}</span>
            </div>
            """
            iframe = folium.IFrame(html=popup_html, width=200, height=200)
            popup = folium.Popup(iframe, max_width=250)

            # Add marker
            folium.Marker(
                location=[lat, lon],
                popup=popup,
                icon=folium.Icon(color="blue", icon="user")
            ).add_to(m)
        except Exception as e:
            print(f"Skipping row due to error: {e}")

# 6. Save to HTML file
m.save("telangana_mla_map.html")
#pd.DataFrame(Marker_coordinates).to_csv("telangana_constituency_centroids.csv", index=False)
print("✅ Map saved as telangana_mla_map.html")


✅ Map saved as telangana_mla_map.html


In [30]:
# 1.5. Load district boundaries
with open("TS_District_Boundary_33_FINAL.geojson", "r", encoding="utf-8") as f:
    new_districts_geojson = json.load(f)

In [31]:
new_districts_geojson

{'type': 'FeatureCollection',
 'name': 'TS_District_Boundary_33_FINAL',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'Shape_Leng': 397121.089982,
    'STATE': 'Telangana',
    'DISTRICT_N': 'Adilabad',
    'Shape_Le_1': 397121.089982,
    'Shape_Area': 3980340811.38},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[78.33813931402472, 19.88297371580062],
      [78.33813950393258, 19.882973717738345],
      [78.33832543905994, 19.8829872048681],
      [78.33890678513241, 19.883029369714784],
      [78.33934750654994, 19.883061336380898],
      [78.33974147568968, 19.88307173281271],
      [78.34077801359363, 19.88309908597109],
      [78.34092977162717, 19.883068321863455],
      [78.34126324203717, 19.883000723442173],
      [78.34151210761205, 19.882950272979667],
      [78.3418343490029, 19.88287445156311],
      [78.34244679718286, 19.882730345593217],
      [78.34295668774595, 19.882